In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
y = pd.read_csv('data/train.csv')


In [3]:
def download_data(signs):
    !echo "" > 'junk_out'
    old_sign = ''
    old_pid = None
    for i, sign in enumerate(signs):
        for participant_id in y.participant_id.unique():
            y_subset = y.loc[(y.sign == sign) & (y.participant_id == participant_id)]
            data = []
            for path in y_subset.path:
                pid_dir = '/'.join(path.split('/')[:-1])
                old_path_to_parquet = "zip_dir/" + path.split('/')[-1]
                path_to_zip_file = old_path_to_parquet + ".zip"
                !kaggle competitions download -c asl-signs -f $path -p 'zip_dir'
                !unzip -n -qq $path_to_zip_file -d 'zip_dir' 2>> junk_out
                !mv $old_path_to_parquet $path 2>> junk_out
                !rm $path_to_zip_file

def release_data():
    !find zip_dir -type f -delete
    !find train_landmark_files -type f -delete

def update_dataset(dataset, signs, col_names):

    for i, sign in enumerate(signs):
        print(f'sign: {sign}, {i+1}/{len(signs)}')
        for j, participant_id in enumerate(y.participant_id.unique()):
            print(f'    pid: {j+1}/21')
            participant_id = 26734
            y_subset = y.loc[(y.sign == sign) & (y.participant_id == participant_id)]
            data = []
            for path in y_subset.path:
                df = pd.read_parquet(path)
                this_data = []
                for type in set(df.type):
                    df_t = df.loc[df.type == type]
                    for lm_i in set(df_t.landmark_index):
                        df_t_i = df_t.loc[(df_t['x'].notna()) & (df_t['y'].notna()) & (df_t.landmark_index == lm_i)]
                        if df_t_i.shape[0] != 0:
                            this_data.append(df_t_i.iloc[0]['x'])
                            this_data.append(df_t_i.iloc[0]['y'])
                            this_data.append(df_t_i.iloc[-1]['x'] - df_t_i.iloc[0]['x'])
                            this_data.append(df_t_i.iloc[-1]['y'] - df_t_i.iloc[0]['y'])
                        else:
                            this_data.extend([0,0,0,0])
                data.append(this_data + [sign])
            sign_df = pd.DataFrame(data, columns=col_names + ['sign'])
            pd.concat((dataset,sign_df), axis=0)
    return dataset

In [4]:
col_names = pd.read_csv('data/col_names.csv', index_col=0)['0']
dataset = pd.DataFrame([],columns=col_names)

signs = ['blow','drink','donkey','finish','up']

download_data(signs)
dataset = update_dataset(dataset,signs,col_names)
release_data()

^C
User cancelled operation
rm: zip_dir/1000035562.parquet.zip: No such file or directory


In [ ]:
dataset